In [1]:
import requests
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/ksenia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Сбор корпуса

Мы решили построить корпус на основе стихотворений советский поэтов с сайта: https://rustih.ru/stixi-sovetskix-poetov/

In [3]:
session = requests.session()
poets, titles, poems = [], [], []

In [4]:
def get_poem(poem_url):
  poem_req = session.get(poem_url)
  poem_page = poem_req.text
  poem_soup = BeautifulSoup(poem_page)
  poem = BeautifulSoup(str(poem_soup.find('div', {'class': 'entry-content poem-text'})).split('<h2>')[0]).text
  if len(poem.split()) >= 110:
    return poem
  return False

In [5]:
def get_poet(poet_url):
  poet_req = session.get(poet_url)
  poet_page = poet_req.text
  poet_soup = BeautifulSoup(poet_page)
  all_poems = poet_soup.find_all('div', {'class': 'entry-title'})
  global poets
  global titles
  global poems
  for poem_url in all_poems:
    poem = get_poem(poem_url.find('a', href=True)['href'])
    if poem != False:
      try:
        poet, title = poem_url.text.strip().split(' — ')
        poets.append(poet)
        titles.append(title)
        poems.append(poem)
      except:
        print(poem_url.text)
  return poets, titles, poems

In [6]:
def get_data(url):
  req = session.get(url)
  page = req.text
  soup = BeautifulSoup(page)
  article = soup.find('article')
  links = article.find_all('li')
  for link in tqdm(links[:7]):
    get_poet(link.find('a', href=True)['href'])

In [7]:
get_data('https://rustih.ru/stixi-sovetskix-poetov/')

 14%|█▍        | 1/7 [00:09<00:55,  9.25s/it]

Марина Цветаева — Асе (Ты — принцесса из царства не светского)


 29%|██▊       | 2/7 [00:17<00:44,  8.92s/it]

Сергей Есенин — Да! Теперь — решено. Без возврата


100%|██████████| 7/7 [01:19<00:00, 11.31s/it]


В результате у нас получилось 263 стихотворений длиной не меньше 110 токенов. 

Стихотворения были написаны 7 поэтами: Марина Цветаева, Иван Бунин, Владимир Маяковский, Валерий Брюсов, Сергей Есенин, Анна Ахматова и Александр Блок.

Предложения из одного слова мы решили удалить, так как они не имеют никакой практической значимости.

In [8]:
len_poems = []
for poem in poems:
  len_poems.append(len(poem.split()))

In [9]:
len(poems), min(len_poems)

(263, 110)

In [10]:
df = pd.DataFrame(columns=['poet', 'title', 'sentence'])

In [11]:
def parse_poem(poem):
  sentences = sent_tokenize(poem)
  clean_sentences = []
  for sent in sentences:
    clean_sent = [word.strip() for word in word_tokenize(sent) if word.strip().isalpha()]
    if len(clean_sent) > 1:
      clean_sentences.append(' '.join(clean_sent))
  return clean_sentences

In [12]:
for i, poet in enumerate(poets):
  poem = poems[i]
  clean_poem = parse_poem(poem)
  for sent in clean_poem:
    df = df.append({'poet': poet, 'title': titles[i], 'sentence': sent}, ignore_index=True)

In [13]:
df.head()

,poet,title,sentence
0,Иван Бунин,Одиночество,И ветер и дождик и мгла Над холодной пустыней ...
1,Иван Бунин,Одиночество,Здесь жизнь до весны умерла До весны опустели ...
2,Иван Бунин,Одиночество,Я на даче один
3,Иван Бунин,Одиночество,Мне темно За мольбертом и дует в окно
4,Иван Бунин,Одиночество,Вчера ты была у меня Но тебе уж тоскливо со мной


In [14]:
df['poet'].unique()

array(['Иван Бунин', 'Марина Цветаева', 'Сергей Есенин',
       'Владимир Маяковский', 'Валерий Брюсов', 'Александр Блок',
       'Анна Ахматова'], dtype=object)

## Сбор корпуса

Мы решили сделать базу данных на основе SQL, чтобы разделить метаданные стихотворений и необходимые для поиска токены с предложениями.

In [15]:
import sqlite3

In [16]:
conn = sqlite3.connect('poems_corpus.db')
cur = conn.cursor()

На данном этапе БД состоит из трёх таблиц:
- info: информация о поэте и названии стихотворения
- sentences: очищенные предложения
- poems_to_info: промежуточная таблица, соединяющая метаданные с предложениями

In [17]:
cur.execute("""
CREATE TABLE IF NOT EXISTS info (
    id_info INTEGER PRIMARY KEY, 
    poet TEXT,
    title TEXT
)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS sentences (
    id_sent INTEGER PRIMARY KEY, 
    sent TEXT
)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS poems_to_info
(id INTEGER PRIMARY KEY AUTOINCREMENT, id_info int, id_sent int) 
""")

In [18]:
new_df = df.groupby(['title', 'poet'])['sentence'].agg(list).reset_index()

In [19]:
cur.execute('SELECT id_info, poet, title FROM info')
db_info = {}
max_id = 0
for idx, poet, title in cur.fetchall():
    max_id += 1
    if poet in db_info:
        db_info[poet].append(title)
    else:
        db_info[poet] = [title]

cur.execute('SELECT COUNT(id_sent) FROM sentences')
poem_cnt = cur.fetchone()[0] + 1
for index, row in new_df.iterrows():
    poet = row['poet']
    title = row['title']
    sentences = row['sentence']
    if poet not in db_info:
        db_info[poet] = []
    
    if title not in db_info[poet]:
        db_info[poet].append(title)
        max_id += 1
        cur.execute('INSERT INTO info VALUES (?, ?, ?)', (max_id, poet, title))
        conn.commit()
        for poem_sent in sentences:
            cur.execute('INSERT INTO sentences VALUES (?, ?)', (poem_cnt, poem_sent))
            cur.execute('INSERT INTO poems_to_info (id_info, id_sent) VALUES (?, ?)', (max_id, poem_cnt))
            poem_cnt += 1
            conn.commit()

In [20]:
cur.execute('SELECT COUNT(id_sent) FROM sentences')
print(cur.fetchone()[0])

8415
